In [1]:
# imports
import requests
import pandas as pd
import os
import json

In [2]:
apikey = os.environ['FOURSQUARE_API_KEY']

List all business in the surounding Vancouver Area

In [7]:
business_ids = []

def get_business_ids(station_id):
    url = "https://api.foursquare.com/v3/places/search"
    
    params = {
        "ll": str(coords),
        "radius": "50",
    }
    
    headers = {
        "Accept": "application/json",
        "Authorization": os.environ['FOURSQUARE_API_KEY']
    }
    
    # Get the request
    response = requests.get(url, params=params, headers=headers)
    response 
    
    # Convert to Python object with .json()
    res = response.json()
    res
    
    # Create a dictionary for each result
    for i in range(len(res['results'])):
        location = {
            'station_id': station_id,
            'fsq_id': res['results'][i]['fsq_id'],
            'fsq_lat': res['results'][i]['geocodes']['main']['latitude'],
            'fsq_long': res['results'][i]['geocodes']['main']['longitude']
        }
        business_ids.append(location)


In [8]:
# Data file
data_file = "station_data.csv"

# Read the data file
df = pd.read_csv(data_file)

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    coords = str(row['Latitude']) + "," + str(row['Longitude'])
    station_id = row['station_id']
    bikes_available = row['Bikes_Available']
    get_business_ids(station_id)

# Create a list of dictionaries
location_dicts = []
for location in business_ids:
    location_dict = {
            'station_id': station_id,
            'fsq_id': location['fsq_id'],   
            'fsq_lat': location['fsq_lat'],
            'fsq_long': location['fsq_long']
    }
    location_dicts.append(location_dict)


In [9]:
# Convert the list of dictionaries into a DataFrame
poi_locations_df = pd.DataFrame(location_dicts)

# Remove duplicates in the 'fsq_id' column of poi_locations_df
poi_locations_df = poi_locations_df.drop_duplicates(subset='fsq_id')

# Print the updated poi_locations_df 
poi_locations_df

,station_id,fsq_id,fsq_lat,fsq_long
0,13021bba7bbeca1ccd8726ca17da59e6,5d2843a1c56f480023f8e4cd,49.262886,-123.114469
1,13021bba7bbeca1ccd8726ca17da59e6,70ea1bc45d084e85ff3ba366,49.262860,-123.114142
2,13021bba7bbeca1ccd8726ca17da59e6,4ca6bd5644a8224b82bb0640,49.274594,-123.121875
3,13021bba7bbeca1ccd8726ca17da59e6,4c48aa9119fde21ef3720c76,49.274419,-123.121617
4,13021bba7bbeca1ccd8726ca17da59e6,4aa80cc4f964a520fd4e20e3,49.274337,-123.122178
...,...,...,...,...
1207,13021bba7bbeca1ccd8726ca17da59e6,f24909f5aa13436298902e9b,49.250729,-123.101269
1208,13021bba7bbeca1ccd8726ca17da59e6,e99715bb54d3405dd72ad9f7,49.251162,-123.101224
1209,13021bba7bbeca1ccd8726ca17da59e6,4cafd9a3c5e6a1cd1983d0f6,49.251388,-123.101244
1210,13021bba7bbeca1ccd8726ca17da59e6,4d507fab80df88bf5c4cf197,49.251218,-123.101229


In [11]:
# create a list of point of interest information
poi_info_list = []

# get fsq_id,id,name,rating,popularity,location
for f_id in poi_locations_df['fsq_id']:
    
    url = "https://api.foursquare.com/v3/places/" + f_id

    params = {
        "fields": "fsq_id,name,rating,popularity,location,categories"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": os.environ['FOURSQUARE_API_KEY']
    }

    response = requests.request("GET", url, params=params, headers=headers)
    res = response.json()
    poi_info_list.append(res)

In [29]:
# create dataframe
parks_df = pd.json_normalize(poi_info_list)

# show df
parks_df.head()

,fsq_id,categories,name,popularity,rating,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region,location.address_extended,location.po_box
0,5d2843a1c56f480023f8e4cd,"[{'id': 13059, 'name': 'Juice Bar', 'icon': {'...",Jugo Juice,0.110323,6.4,496 West Broadway,CA,,"496 West Broadway, Vancouver BC V5Y 1R3",Vancouver,V5Y 1R3,BC,NaN,NaN
1,70ea1bc45d084e85ff3ba366,"[{'id': 15000, 'name': 'Health and Medicine', ...",Screening Mammography Program of B C,NaN,NaN,5800 Mtn View Blvd,CA,NaN,"5800 Mtn View Blvd, Delta BC V4K 3V6",Delta,V4K 3V6,BC,NaN,NaN
2,4ca6bd5644a8224b82bb0640,"[{'id': 16047, 'name': 'Sculpture Garden', 'ic...",Equestrian Monument,0.297127,NaN,1174 Mainland,CA,Davie and mainland,"1174 Mainland (Davie and mainland), Vancouver BC",Vancouver,NaN,BC,NaN,NaN
3,4c48aa9119fde21ef3720c76,"[{'id': 17089, 'name': 'Gift Store', 'icon': {...",Terra Cotta Warriors,0.450186,NaN,561 Columbia St,CA,Keefer Street,"561 Columbia St (Keefer Street), Vancouver BC ...",Vancouver,V6A 4H5,BC,NaN,NaN
4,4aa80cc4f964a520fd4e20e3,"[{'id': 13276, 'name': 'Sushi Restaurant', 'ic...",Sushi Maro Japanese Restaurant,0.912786,6.1,220 Davie St,CA,btwn Mainland & Pacific Blvd,"220 Davie St (btwn Mainland & Pacific Blvd), V...",Vancouver,V6B 5Z6,BC,NaN,NaN


In [55]:
# rename column
parks_df = parks_df.rename(columns={'location.formatted_address':'address'})

# drop columns
parks_df = parks_df.drop(columns=['location.address', 'location.country', 'location.cross_street', 'location.locality', 'location.postcode', 'location.region', 'location.address_extended', 'location.po_box'])

# show df
parks_df.head()

,fsq_id,categories,name,popularity,rating,address
0,5d2843a1c56f480023f8e4cd,"[{'id': 13059, 'name': 'Juice Bar', 'icon': {'...",Jugo Juice,0.110323,6.4,"496 West Broadway, Vancouver BC V5Y 1R3"
1,70ea1bc45d084e85ff3ba366,"[{'id': 15000, 'name': 'Health and Medicine', ...",Screening Mammography Program of B C,NaN,NaN,"5800 Mtn View Blvd, Delta BC V4K 3V6"
2,4ca6bd5644a8224b82bb0640,"[{'id': 16047, 'name': 'Sculpture Garden', 'ic...",Equestrian Monument,0.297127,NaN,"1174 Mainland (Davie and mainland), Vancouver BC"
3,4c48aa9119fde21ef3720c76,"[{'id': 17089, 'name': 'Gift Store', 'icon': {...",Terra Cotta Warriors,0.450186,NaN,"561 Columbia St (Keefer Street), Vancouver BC ..."
4,4aa80cc4f964a520fd4e20e3,"[{'id': 13276, 'name': 'Sushi Restaurant', 'ic...",Sushi Maro Japanese Restaurant,0.912786,6.1,"220 Davie St (btwn Mainland & Pacific Blvd), V..."


In [31]:
# view categories dictionary
parks_df['categories']

0       [{'id': 13059, 'name': 'Juice Bar', 'icon': {'...
1       [{'id': 15000, 'name': 'Health and Medicine', ...
2       [{'id': 16047, 'name': 'Sculpture Garden', 'ic...
3       [{'id': 17089, 'name': 'Gift Store', 'icon': {...
4       [{'id': 13276, 'name': 'Sushi Restaurant', 'ic...
                              ...                        
1205    [{'id': 17010, 'name': 'Used Car Dealership', ...
1206    [{'id': 11002, 'name': 'Agriculture and Forest...
1207    [{'id': 13352, 'name': 'Thai Restaurant', 'ico...
1208    [{'id': 15007, 'name': 'Dentist', 'icon': {'pr...
1209    [{'id': 17043, 'name': 'Clothing Store', 'icon...
Name: categories, Length: 1210, dtype: object

In [46]:
# normalize categories
categories_df = pd.json_normalize(parks_df['categories'])

# rename column
categories_df = categories_df.rename(columns={0: 'categories'})

# drop columns
categories_df = categories_df.drop(columns=[1,2])

# view results
categories_df.head()

,categories
0,"{'id': 13059, 'name': 'Juice Bar', 'icon.prefi..."
1,"{'id': 15000, 'name': 'Health and Medicine', '..."
2,"{'id': 16047, 'name': 'Sculpture Garden', 'ico..."
3,"{'id': 17089, 'name': 'Gift Store', 'icon.pref..."
4,"{'id': 13276, 'name': 'Sushi Restaurant', 'ico..."


In [51]:
# normalize categories
cat_df = pd.json_normalize(categories_df['categories'])

# drop columns
cat_df = cat_df.drop(columns=['icon.prefix','icon.suffix'])

#view results
cat_df

,id,name
0,13059.0,Juice Bar
1,15000.0,Health and Medicine
2,16047.0,Sculpture Garden
3,17089.0,Gift Store
4,13276.0,Sushi Restaurant
...,...,...
1205,17010.0,Used Car Dealership
1206,11002.0,Agriculture and Forestry Service
1207,13352.0,Thai Restaurant
1208,15007.0,Dentist


In [63]:
# concat dataframe
resto_df = pd.concat([parks_df[['fsq_id']], cat_df, pd.DataFrame(parks_df[['popularity', 'rating', 'address']])], axis=1)

resto_df.head()

,fsq_id,id,name,popularity,rating,address
0,5d2843a1c56f480023f8e4cd,13059.0,Juice Bar,0.110323,6.4,"496 West Broadway, Vancouver BC V5Y 1R3"
1,70ea1bc45d084e85ff3ba366,15000.0,Health and Medicine,NaN,NaN,"5800 Mtn View Blvd, Delta BC V4K 3V6"
2,4ca6bd5644a8224b82bb0640,16047.0,Sculpture Garden,0.297127,NaN,"1174 Mainland (Davie and mainland), Vancouver BC"
3,4c48aa9119fde21ef3720c76,17089.0,Gift Store,0.450186,NaN,"561 Columbia St (Keefer Street), Vancouver BC ..."
4,4aa80cc4f964a520fd4e20e3,13276.0,Sushi Restaurant,0.912786,6.1,"220 Davie St (btwn Mainland & Pacific Blvd), V..."


In [66]:
resto_df.shape

(1210, 6)

In [72]:
# Assuming 'id' is a column in your DataFrame
filtered_df = resto_df[~resto_df['id'].between(10070, 12126)]
filtered_df = resto_df[~resto_df['id'].between(13400, 15500)]
filtered_df = resto_df[~resto_df['id'].between(16080, 17999)]

# Print the updated DataFrame
filtered_df.head()

,fsq_id,id,name,popularity,rating,address
0,5d2843a1c56f480023f8e4cd,13059.0,Juice Bar,0.110323,6.4,"496 West Broadway, Vancouver BC V5Y 1R3"
1,70ea1bc45d084e85ff3ba366,15000.0,Health and Medicine,NaN,NaN,"5800 Mtn View Blvd, Delta BC V4K 3V6"
2,4ca6bd5644a8224b82bb0640,16047.0,Sculpture Garden,0.297127,NaN,"1174 Mainland (Davie and mainland), Vancouver BC"
4,4aa80cc4f964a520fd4e20e3,13276.0,Sushi Restaurant,0.912786,6.1,"220 Davie St (btwn Mainland & Pacific Blvd), V..."
6,5a878c33f8cbd46ab822ef69,13377.0,Vegan and Vegetarian Restaurant,0.972816,8.3,"1165 Mainland St, Vancouver BC V6B 5P2"


In [74]:
# merged poi_locations_df and parks_df
merged_fsq_df = pd.merge(poi_locations_df, filtered_df, on='fsq_id', how='inner')

# view merged file
merged_fsq_df.head()

,station_id,fsq_id,fsq_lat,fsq_long,id,name,popularity,rating,address
0,13021bba7bbeca1ccd8726ca17da59e6,5d2843a1c56f480023f8e4cd,49.262886,-123.114469,13059.0,Juice Bar,0.110323,6.4,"496 West Broadway, Vancouver BC V5Y 1R3"
1,13021bba7bbeca1ccd8726ca17da59e6,70ea1bc45d084e85ff3ba366,49.262860,-123.114142,15000.0,Health and Medicine,NaN,NaN,"5800 Mtn View Blvd, Delta BC V4K 3V6"
2,13021bba7bbeca1ccd8726ca17da59e6,4ca6bd5644a8224b82bb0640,49.274594,-123.121875,16047.0,Sculpture Garden,0.297127,NaN,"1174 Mainland (Davie and mainland), Vancouver BC"
3,13021bba7bbeca1ccd8726ca17da59e6,4aa80cc4f964a520fd4e20e3,49.274337,-123.122178,13276.0,Sushi Restaurant,0.912786,6.1,"220 Davie St (btwn Mainland & Pacific Blvd), V..."
4,13021bba7bbeca1ccd8726ca17da59e6,5a878c33f8cbd46ab822ef69,49.274991,-123.121909,13377.0,Vegan and Vegetarian Restaurant,0.972816,8.3,"1165 Mainland St, Vancouver BC V6B 5P2"


In [76]:
# create csv file for our model
merged_fsq_df.to_csv('merged_fsq_df.csv', index=False)